# PREDICCIÓN COSTE TRASPASO JUGADORES

# 1. IMPORTAMOS LIBRERÍAS Y LEEMOS EL CSV

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("transfers.csv")

In [ ]:
df.head(10)

### Información de la base de datos

In [ ]:
#VEMOS SI HAY VALORES NULOS
df.isna().any()

In [ ]:
df.info()

In [ ]:
df.describe

## 2. AJUSTES BASE DE DATOS

### Hacemos una copia del original para no modificarlo

In [ ]:
df_clean=df.copy()

### Eliminación de filas con: valores nulos en traspasos, retiradas o agentes libre

In [ ]:
#BUSCAMOS EL VALOR ASOCIADO AL CLUB "RETIRADO"" PARA ELIMINARLO
df.query("to_club_id <= 123 and to_club_id >= 123")

In [ ]:
#ELIMINAMOS LOS TRANSPASOS QUE SON RETIRADAS
df_clean=df_clean[df_clean["to_club_id"]!=123]

In [ ]:
#BUSCAMOS EL VALOR ASOCIADO AL CLUB "SIN EQUIPO"" PARA ELIMINARLO
df.query("to_club_id <= 515 and to_club_id >= 515")

In [ ]:
#ELIMINAMOS LOS TRANSPASOS QUE SON SIN EQUIPO(NO SON TRASPASOS)
df_clean=df_clean[df_clean["to_club_id"]!=515]

In [ ]:
#ELIMINAMOS FILAS QUE NO TIENEN LA CANTIDAD DE TRASPASO
df_clean=df_clean[df_clean["transfer_fee"].notna()]

### Eliminación Características

In [ ]:
#ELIMINAMOS ATRIBUTOS DUPLICADOS Y TRANSFER_DATE, AUNQUE PODRÍA SER INTERESANTE
del df_clean["from_club_name"]
del df_clean["to_club_name"]
del df_clean["player_name"]
del df_clean["transfer_date"]

### Conversión de características a numérico

In [ ]:
#CONVERTIMOS CATEGÓRICO A NUMÉRICO
df_clean=pd.get_dummies(df_clean,dtype='int')

In [ ]:
df_clean

In [ ]:
#VEMOS YA QUE TODAS LAS CATEGORIAS SON  NUMERICAS
df_clean.info()

### Completamos valores nulos con 

In [ ]:
#VEMOS ANALISIS DE CATEGORIAS
#df_clean.describe()

In [ ]:
#mediana_market_value=df_clean["market_value_in_eur"].median()

In [ ]:
#df_clean["market_value_in_eur"]=df_clean["market_value_in_eur"].fillna(mediana_market_value)

In [ ]:
#df_clean

In [ ]:
#ELIMINAMOS FILAS QUE NO TIENEN LA CANTIDAD DE TRASPASO
df_clean=df_clean[df_clean["market_value_in_eur"].notna()]

In [ ]:
df_clean

# 3. Correlación entre características

In [ ]:
# Mostrar la correlación entre los atributos del conjunto de datos
corr_matrix = df_clean.corr()
corr_matrix["transfer_fee"].sort_values(ascending=False)

## Predicción con regresión lineal simple a partir del market value

## Particionado del conjunto de datos

In [ ]:
from sklearn.model_selection import train_test_split
# Construcción de una función que realice el particionado completo
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [ ]:
df_reg=df_clean[["transfer_fee","market_value_in_eur"]]/1000000
train_set, val_set, test_set = train_val_test_split(df_reg)

In [ ]:
X_train_set=train_set.drop("transfer_fee",axis=1)
y_train_set=train_set["transfer_fee"]
X_val_set=val_set.drop("transfer_fee",axis=1)
y_val_set=val_set["transfer_fee"]
X_test_set=test_set.drop("transfer_fee",axis=1)
y_test_set=test_set["transfer_fee"]

In [ ]:
y_train_set.describe()

In [ ]:
# Representación gráfica del conjunto de datos
plt.plot(df_reg['market_value_in_eur'], df_reg['transfer_fee'], "b.")
plt.xlabel("Valor Market")
plt.ylabel("Coste Traspaso")
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
# Construcción del modelo y ajuste de la función hipótesis
lin_reg = LinearRegression()
lin_reg.fit(df_reg['market_value_in_eur'].values.reshape(-1, 1), df_reg['transfer_fee'].values)

In [ ]:
# Predicción para el valor mínimo y máximo del conjunto de datos de entrenamiento
X_min_max = np.array([[df_reg["market_value_in_eur"].min()], [df_reg["market_value_in_eur"].max()]])
print(X_min_max)
y_train_pred = lin_reg.predict(X_min_max)
print(y_train_pred)

In [ ]:
# Representación gráfica de la función hipótesis generada
plt.plot(X_min_max, y_train_pred, "g-")
plt.plot(df_reg['market_value_in_eur'], df_reg['transfer_fee'], "b.")
plt.xlabel("Valor Market")
plt.ylabel("Coste Traspaso")
plt.show()

In [ ]:
x_new = np.array([[120]]) # Isak 

# Predicción del coste que tendría el traspaso
coste = lin_reg.predict(x_new) 

print("El coste del traspaso de Isak sería:", int(coste[0]), "millones de euros")

In [ ]:
import plotly.express as px

# Filter out rows where either transfer_fee or market_value_in_eur is zero or null
scatter_data = df[(df['transfer_fee'] > 25000000) & (df['market_value_in_eur'] > 25000000)]

# Create an interactive scatter plot with a trendline (correlation line)
fig = px.scatter(
    scatter_data, 
    x='market_value_in_eur', 
    y='transfer_fee', 
    title='Interactive Scatterplot: Transfer Fee vs Market Value in EUR',
    labels={'market_value_in_eur': 'Market Value (in EUR)', 'transfer_fee': 'Transfer Fee (in EUR)'},
    hover_data=['player_name', 'transfer_season'],
    trendline='ols'  # Add Ordinary Least Squares (OLS) trendline
)

# Show the plot
fig.show()

In [ ]:
 #Calculate the correlation between transfer_fee and market_value_in_eur
correlation = scatter_data['transfer_fee'].corr(scatter_data['market_value_in_eur'])
print(f'Correlation between Transfer Fee and Market Value: {correlation:.4f}')

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred=np.array(df_reg["market_value_in_eur"])
y_train_pred = lin_reg.predict(y_pred.reshape(-1,1))
print(y_train_pred)
rmse=np.sqrt(mean_squared_error(df_reg["transfer_fee"],y_train_pred))
rmse

## Regresión Lineal Múltiple

In [ ]:
x_clean= df_clean.copy()
del x_clean["transfer_fee"]
x_clean

In [ ]:
# Construcción del modelo y ajuste de la función hipótesis
lin_reg = LinearRegression()

lin_reg.fit(x_clean, df_clean['transfer_fee'])

In [ ]:
#.values.reshape(-1, 1

In [ ]:
y_train_pred = lin_reg.predict(x_clean)
rmse=np.sqrt(mean_squared_error(df_reg["transfer_fee"],y_train_pred))
rmse

## Modelo Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=100,max_depth=5)


In [ ]:
df_clean["transfer_fee"]=df_clean["transfer_fee"]/100
df_clean["market_value_in_eur"]=df_clean["market_value_in_eur"]/100

train_set, val_set, test_set = train_val_test_split(df_clean)


In [ ]:
X_train_set=train_set.drop("transfer_fee",axis=1)
y_train_set=train_set["transfer_fee"]
X_val_set=val_set.drop("transfer_fee",axis=1)
y_val_set=val_set["transfer_fee"]
X_test_set=test_set.drop("transfer_fee",axis=1)
y_test_set=test_set["transfer_fee"]

In [ ]:
model.fit(X_train_set,y_train_set)

In [ ]:

y_val_pred = lin_reg.predict(X_val_set)
rmse=np.sqrt(mean_squared_error(y_val_set,y_val_pred))
rmse

In [ ]:
# Random Forest plot
y_test=y_val_set
y_pred_rf=y_val_pred
std_y = np.std(y_test)
plt.figure(figsize=(14, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.5, color="blue",ec='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2,label="perfect model")
plt.plot([y_test.min(), y_test.max()], [y_test.min() + std_y, y_test.max() + std_y], 'r--', lw=1, label="+/-1 Std Dev")
plt.plot([y_test.min(), y_test.max()], [y_test.min() - std_y, y_test.max() - std_y], 'r--', lw=1, )
plt.ylim(0,6)
plt.title("Random Forest Predictions vs Actual")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:

# Representación gráfica de la función hipótesis generada
y_val_set=y_val_set
y_val_pred=y_val_pred
plt.plot(y_val_pred,y_val_set, "g-")
plt.xlabel("coste predecido")
plt.ylabel("Coste Traspaso")
plt.show()